In [1]:
import csv
from string import punctuation
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical
import math

Using TensorFlow backend.


In [4]:
fileName = 'SampleTweets.csv'
tweets = pd.read_csv(fileName, encoding = "ISO-8859-1")

In [8]:
tweets.head()

,sentiment,date,text,score
0,|neutral|,5/19/17,RT @Sandro_power: $AAPL $BABA will hold these...,0.0
1,NaN,NaN,NaN,NaN
2,|positive|,5/19/17,RT @mytradingbuddy: #Stocks #Traders LEARN Th...,2.0
3,NaN,NaN,NaN,NaN
4,|neutral|,5/19/17,AppleÛªs $AAPL Nokia Suit More Pressing Than...,0.0


In [11]:
tweets = tweets.dropna()


In [17]:
print(tweets.score)

0      0.0
2      2.0
4      0.0
6      5.0
8      0.0
10    -3.0
12     0.0
14     2.0
16    -5.0
18     2.0
20     0.0
22    -3.0
24     0.0
26     1.0
28     1.0
30    -1.0
32     1.0
34     0.0
36     0.0
38     1.0
40    -1.0
42     0.0
44     0.0
46     0.0
48    -2.0
50     0.0
52    -2.0
54     0.0
56     2.0
58     0.0
      ... 
918    1.0
920    0.0
922    0.0
924    3.0
926    0.0
928    2.0
930    1.0
932    3.0
934    0.0
936    0.0
938    0.0
940    3.0
942    0.0
944    0.0
946    0.0
948   -1.0
950    2.0
952    0.0
954    1.0
956    0.0
958    2.0
960    2.0
962   -1.0
964    2.0
966    1.0
968   -1.0
970   -1.0
972   -1.0
974   -1.0
976    0.0
Name: score, Length: 489, dtype: float64


In [18]:
stop_words = stopwords.words('english')

def preprocess_text(text):
    text = text.lower()
    text_clean = ""
    for w in text.split():
        if w not in stop_words:
            text_clean += w + " "
    return ''.join([c for c in text_clean if c not in punctuation])


In [19]:
tweets["text"] = tweets["text"].apply(preprocess_text)



In [20]:
tokenizer = Tokenizer(num_words=25000, lower=True,split=' ')
tokenizer.fit_on_texts(tweets['text'].values)
X = tokenizer.texts_to_sequences(tweets['text'].values)
X = pad_sequences(X)

In [22]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(25000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/Users/ruben/anaconda3/envs/snowflakes/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/Users/ruben/anaconda3/envs/snowflakes/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 128)           3200000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 603       
Total params: 3,463,803
Trainable params: 3,463,803
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
Y = pd.get_dummies(tweets['sentiment']).values
X_train, X_valid, Y_train, Y_valid = train_test_split(X,Y, test_size = 0.20, random_state = 36)

#Here we train the Network.

model.fit(X_train, Y_train, batch_size =batch_size, epochs = 4,  verbose = 5)

Epoch 1/4
Epoch 2/4
Epoch 3/4
Epoch 4/4


In [26]:
score,acc = model.evaluate(X_valid, Y_valid, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.51
Validation Accuracy: 0.79


In [27]:
pred = model.predict(X_valid, batch_size=batch_size, verbose=0, steps=None)


In [21]:
X_valid.shape

(761, 21)

In [22]:
pred.shape

(761, 3)

In [28]:
predicted_texts = tokenizer.sequences_to_texts(X_valid)

In [29]:
len(predicted_texts)
predicted_texts = np.asarray(predicted_texts)
predicted_texts.shape

(98,)

In [30]:
pred_data = pd.DataFrame(data=np.column_stack((pred, predicted_texts.T), ))

In [31]:
pred_data

,0,1,2,3
0,0.008634582,0.0043449653,0.9870205,cop intc ete cat pru pfe dis hum epd next big ...
1,0.0017635915,0.00058331795,0.99765307,join robinhoodapp well get share stock like aa...
2,0.07606073,0.28027213,0.64366716,see it hedge fund dumps massive blocks aapl ib...
3,0.12080771,0.6682128,0.21097946,rt optionshawk buffett doubled aapl stake
4,0.03100793,0.07431504,0.89467704,berkshire boosts apple stake exits twentyfirst...
5,0.23956384,0.29713124,0.46330497,sjosephburns alright tell like aapl ù÷û
6,1.4044796e-05,2.6774218e-05,0.99995923,rt wallstjesus ùó´ todays active flow gtgt amd...
7,1.4044796e-05,2.6774218e-05,0.99995923,rt wallstjesus ùó´ todays active flow gtgt amd...
8,0.002596966,0.97934204,0.018061114,hightech usbased transnational companies tncs ...
9,0.6191773,0.119322225,0.26150045,ios 11 concept imagines overhauled ipad interf...


In [32]:
X_test = tokenizer.texts_to_sequences(tweets['text'].values)
X_test = pad_sequences(X_test)
print(X_test.shape)


(489, 27)


In [33]:
X_test_scores = model.predict(X_test)


In [34]:
X_test_scores
predicted_texts = tokenizer.sequences_to_texts(X_test)
# pred_data = pd.DataFrame(data=np.column_stack((pred, predicted_texts.T), ))

In [35]:
X_test_scores = pd.DataFrame(data=X_test_scores)
# tweets.merge(X_test_scores,left_index=True, right_index=True)
tweets = pd.merge(tweets, X_test_scores, left_index=True, right_index=True)


In [36]:
tweets

,sentiment,date,text,score,0,1,2
0,|neutral|,5/19/17,rt sandropower aapl baba hold stocks june 16 o...,0.0,0.001692,0.997397,0.000911
2,|positive|,5/19/17,rt mytradingbuddy stocks traders learn 6 indiv...,2.0,0.007552,0.984670,0.007778
4,|neutral|,5/19/17,appleûªs aapl nokia suit pressing qualcomm qc...,0.0,0.000816,0.998633,0.000552
6,|positive|,5/19/17,validate organic strategies httpstcosom8rjvubz...,5.0,0.020948,0.958246,0.020806
8,|neutral|,5/19/17,rt kirillklip investorintel international lith...,0.0,0.205599,0.555674,0.238727
10,|negative|,5/19/17,rt sptrades use stoploss orders protect profit...,-3.0,0.008654,0.982485,0.008861
12,|neutral|,5/19/17,aapl cash floods tech funds fastest clip 15 ye...,0.0,0.011128,0.975391,0.013482
14,|positive|,5/19/17,stocks traders learn 6 individual components i...,2.0,0.030268,0.010930,0.958801
16,|negative|,5/19/17,rt eatsleepntrade may 17 2017 go day history w...,-5.0,0.010010,0.005067,0.984923
18,|positive|,5/19/17,thekl276 thank rt happyfriday see vista partne...,2.0,0.004167,0.992397,0.003437


In [46]:

import pandas_datareader.data as web
import datetime as dt

start = dt.datetime(2017, 5, 17)
end =  dt.datetime(2017,5, 19) #dt.datetime.now() 

df_stock = web.DataReader('AAPL', 'yahoo', start, end)

In [47]:
df_stock['Pct_change'] = (df_stock['Close'] - df_stock['Open']) / df_stock['Open'] * 100.0


In [48]:
tweets['timestamp'] = pd.to_datetime(tweets['date'])
tweets['timestamp']= tweets['timestamp'].dt.date

In [49]:
df_daily_mean=(tweets.groupby(tweets.timestamp).mean())


In [50]:
df_daily_mean

,score,0,1,2
timestamp,,,,
2017-05-17,0.392857,0.140484,0.519681,0.339834
2017-05-18,0.760000,0.084587,0.582057,0.333357
2017-05-19,0.584270,0.112381,0.472385,0.415234


In [51]:
df_full = pd.concat([df_stock[['High', 'Low', 'Open', 'Adj Close', 'Pct_change']],\
                     df_daily_mean], axis=1, sort=False)






In [52]:
df_full

,High,Low,Open,Adj Close,Pct_change,score,0,1,2
2017-05-17,154.570007,149.710007,153.600006,146.293030,-2.180993,0.392857,0.140484,0.519681,0.339834
2017-05-18,153.339996,151.130005,151.270004,148.522705,0.839551,0.760000,0.084587,0.582057,0.333357
2017-05-19,153.979996,152.630005,153.380005,149.029007,-0.208637,0.584270,0.112381,0.472385,0.415234


In [53]:
df_full = df_full[[ 'High', 'Low', 'Open', 'Adj Close', 'Pct_change',0, 1, 2 ]].interpolate(method='linear', limit_direction='forward', axis=0)



In [54]:
df_full

,High,Low,Open,Adj Close,Pct_change,0,1,2
2017-05-17,154.570007,149.710007,153.600006,146.293030,-2.180993,0.140484,0.519681,0.339834
2017-05-18,153.339996,151.130005,151.270004,148.522705,0.839551,0.084587,0.582057,0.333357
2017-05-19,153.979996,152.630005,153.380005,149.029007,-0.208637,0.112381,0.472385,0.415234


In [62]:
forecast_col = 'Pct_change'

forecast_out = int(math.ceil(0.0333 * len(df_full)))

df_full['Predicted_change'] = df_full[forecast_col].shift(-forecast_out)

In [63]:
df_full

,High,Low,Open,Adj Close,Pct_change,0,1,2,Predicted_change
2015-02-16,NaN,NaN,NaN,NaN,NaN,0.716134,0.247716,0.036150,-1.558339
2015-02-17,49.450001,47.860001,48.770000,46.192921,-1.558339,0.621934,0.192920,0.185147,1.460911
2015-02-18,49.450001,48.180000,48.599998,47.443718,1.460911,0.584268,0.213903,0.201829,-1.969279
2015-02-19,50.799999,49.590000,50.779999,47.895927,-1.969279,0.579342,0.217332,0.203325,2.429229
2015-02-20,51.119999,49.130001,49.810001,49.089001,2.429229,0.593502,0.213430,0.193068,1.336073
2015-02-21,51.480000,49.736668,50.456668,49.182008,1.336073,0.687935,0.170684,0.141380,0.242917
2015-02-22,51.840000,50.343334,51.103334,49.275016,0.242917,0.727824,0.152281,0.119895,-0.850239
2015-02-23,52.200001,50.950001,51.750000,49.368023,-0.850239,0.651518,0.199008,0.149474,2.445813
2015-02-24,51.560001,49.360001,50.290001,49.570084,2.445813,0.627249,0.199985,0.172766,-3.060428
2015-02-25,51.490002,49.459999,51.299999,47.847824,-3.060428,0.627249,0.199985,0.172766,NaN
